In [3]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [4]:
#conditions1 (1800 rpm, 4000 N)

cwd = os.getcwd()
path1 = os.path.join(cwd,r'Bearing1_1\acc_00050.csv')

column_labels1 = ['Hour','Minute','Second','µ_second', 'Horizontal_acceleration', 'Vertical_acceleration'] #determine column labels
acc_c1 = pd.read_csv(path1,names=column_labels1)
print(acc_c1)

path2 = os.path.join(cwd,r'Bearing1_1\temp_00050.csv')
column_labels2 = ['Hour','Minute','Second','X0_x_second', 'Rtd_sensor']
temp_c1 = pd.read_csv(path2,names=column_labels2)
print(temp_c1)

      Hour  Minute  Second  µ_second  Horizontal_acceleration  \
0        9      47      49   65664.0                    0.115   
1        9      47      49   65703.0                   -0.061   
2        9      47      49   65742.0                   -0.019   
3        9      47      49   65781.0                    0.034   
4        9      47      49   65820.0                    0.126   
...    ...     ...     ...       ...                      ...   
2555     9      47      49  165470.0                    0.273   
2556     9      47      49  165510.0                   -0.023   
2557     9      47      49  165550.0                   -0.372   
2558     9      47      49  165580.0                   -0.117   
2559     9      47      49  165620.0                   -0.216   

      Vertical_acceleration  
0                     0.364  
1                    -0.080  
2                    -0.609  
3                     0.271  
4                    -0.011  
...                     ...  
2555     

In [33]:
    import psycopg2
    from io import StringIO

    # Connection parameters
    param_dic = {
        "host"      : "localhost",
        "database"  : "sensordata",
        "user"      : "postgres",
        "password"  : "JNt8hSW4l+MJ]p;.g[WG"
    }

    #Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

    def connect(params_dic):
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            # connect to the PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params_dic)
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            sys.exit(1) 
        print("Connection successful")
        return conn


    def copy_from_stringio(conn, df, table):
        """
        Here we are going save the dataframe in memory 
        and use copy_from() to copy it to the table
        """
        # save dataframe to an in memory buffer
        buffer = StringIO()
        df.to_csv(buffer, index_label='id', header=False)
        buffer.seek(0)

        cursor = conn.cursor()
        try:
            cursor.copy_from(buffer, table, sep=",")
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:

            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("copy_from_stringio() done")
        cursor.close()

In [30]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(acc_c1.keys()) #use column labels for SQL query
string1 = "CREATE TABLE FEMTO_ACC_C1 (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, acc_c1, "FEMTO_ACC_C1") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE FEMTO_ACC_C1 (id int, Hour decimal, Minute decimal, Second decimal, µ_second decimal, Horizontal_acceleration decimal, Vertical_acceleration decimal);
copy_from_stringio() done


In [34]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(temp_c1.keys()) #use column labels for SQL query
string1 = "CREATE TABLE FEMTO_TEMP_C1 (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, temp_c1, "FEMTO_TEMP_C1") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE FEMTO_TEMP_C1 (id int, Hour decimal, Minute decimal, Second decimal, X0_x_second decimal, Rtd_sensor decimal);
copy_from_stringio() done
